<a href="https://colab.research.google.com/github/dantecomedia/Video-Classifcation-using-Deep-Learning/blob/master/test_writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!mkdir "/content/video"

In [ ]:
!cp "/content/gdrive/My Drive/Security Dataset/videos for Test/lHkl9AA0SwKFhu9j.mp4" "/content/video"

In [ ]:
!cp "/content/gdrive/My Drive/Security Dataset/Abuse/weights/Model_Abuse_1.weights.best.hdf5" "/content"
!cp "/content/gdrive/My Drive/Security Dataset/Fighting/weights/Model_Fighting_1.weights.best.hdf5" "/content"
!cp "/content/gdrive/My Drive/Security Dataset/Assault/weights/Model_depthwise_1_Assault.weights.best.hdf5" "/content"
!cp "/content/gdrive/My Drive/Security Dataset/videos for Test/Copy of Abuse023_x264_22.mp4" "/content/video"

In [2]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [3]:
from __future__ import absolute_import
import cv2
import numpy as np 
import pandas
from imutils.video import FPS 
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files
import random
import shutil
#!pip install scikit-video
import numpy as np
from skvideo.io import FFmpegReader, ffprobe
from skvideo.utils import rgb2gray
from PIL import Image
from keras.preprocessing import image
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras import backend as K
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import layers
from keras.engine import InputSpec
from keras.utils import conv_utils
from keras.legacy.interfaces import conv3d_args_preprocessor, generate_legacy_interface
from keras.layers import Conv3D
from keras.backend.tensorflow_backend import _preprocess_padding, _preprocess_conv3d_input

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, GlobalAveragePooling3D
from keras.layers.core import Dense
import tensorflow.keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Dropout
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [4]:

def depthwise_conv3d_args_preprocessor(args, kwargs):
    converted = []

    if 'init' in kwargs:
        init = kwargs.pop('init')
        kwargs['depthwise_initializer'] = init
        converted.append(('init', 'depthwise_initializer'))

    args, kwargs, _converted = conv3d_args_preprocessor(args, kwargs)
    return args, kwargs, converted + _converted

    legacy_depthwise_conv3d_support = generate_legacy_interface(
    allowed_positional_args=['filters', 'kernel_size'],
    conversions=[('nb_filter', 'filters'),
                 ('subsample', 'strides'),
                 ('border_mode', 'padding'),
                 ('dim_ordering', 'data_format'),
                 ('b_regularizer', 'bias_regularizer'),
                 ('b_constraint', 'bias_constraint'),
                 ('bias', 'use_bias')],
    value_conversions={'dim_ordering': {'tf': 'channels_last',
                                        'th': 'channels_first',
                                        'default': None}},
    preprocessor=depthwise_conv3d_args_preprocessor)


class DepthwiseConv3D(Conv3D):
    """Depthwise 3D convolution.
    Depth-wise part of separable convolutions consist in performing
    just the first step/operation
    (which acts on each input channel separately).
    It does not perform the pointwise convolution (second step).
    The `depth_multiplier` argument controls how many
    output channels are generated per input channel in the depthwise step.
    # Arguments
        kernel_size: An integer or tuple/list of 3 integers, specifying the
            depth, width and height of the 3D convolution window.
            Can be a single integer to specify the same value for
            all spatial dimensions.
        strides: An integer or tuple/list of 3 integers,
            specifying the strides of the convolution along the depth, width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
        padding: one of `"valid"` or `"same"` (case-insensitive).
        depth_multiplier: The number of depthwise convolution output channels
            for each input channel.
            The total number of depthwise convolution output
            channels will be equal to `filterss_in * depth_multiplier`.
        groups: The depth size of the convolution (as a variant of the original Depthwise conv)
        data_format: A string,
            one of `channels_last` (default) or `channels_first`.
            The ordering of the dimensions in the inputs.
            `channels_last` corresponds to inputs with shape
            `(batch, height, width, channels)` while `channels_first`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "channels_last".
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you don't specify anything, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        depthwise_initializer: Initializer for the depthwise kernel matrix
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        depthwise_regularizer: Regularizer function applied to
            the depthwise kernel matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        dialation_rate: List of ints.
                        Defines the dilation factor for each dimension in the
                        input. Defaults to (1,1,1)
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
            (see [regularizer](../regularizers.md)).
        depthwise_constraint: Constraint function applied to
            the depthwise kernel matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
    # Input shape
        5D tensor with shape:
        `(batch, depth, channels, rows, cols)` if data_format='channels_first'
        or 5D tensor with shape:
        `(batch, depth, rows, cols, channels)` if data_format='channels_last'.
    # Output shape
        5D tensor with shape:
        `(batch, filters * depth, new_depth, new_rows, new_cols)` if data_format='channels_first'
        or 4D tensor with shape:
        `(batch, new_depth, new_rows, new_cols, filters * depth)` if data_format='channels_last'.
        `rows` and `cols` values might have changed due to padding.
    """

    #@legacy_depthwise_conv3d_support
    def __init__(self,
                 kernel_size,
                 strides=(1, 1, 1),
                 padding='valid',
                 depth_multiplier=1,
                 groups=None,
                 data_format=None,
                 activation=None,
                 use_bias=True,
                 depthwise_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 dilation_rate = (1, 1, 1),
                 depthwise_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 depthwise_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(DepthwiseConv3D, self).__init__(
            filters=None,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            activation=activation,
            use_bias=use_bias,
            bias_regularizer=bias_regularizer,
            dilation_rate=dilation_rate,
            activity_regularizer=activity_regularizer,
            bias_constraint=bias_constraint,
            **kwargs)
        self.depth_multiplier = depth_multiplier
        self.groups = groups
        self.depthwise_initializer = initializers.get(depthwise_initializer)
        self.depthwise_regularizer = regularizers.get(depthwise_regularizer)
        self.depthwise_constraint = constraints.get(depthwise_constraint)
        self.bias_initializer = initializers.get(bias_initializer)
        self.dilation_rate = dilation_rate
        self._padding = _preprocess_padding(self.padding)
        self._strides = (1,) + self.strides + (1,)
        self._data_format = "NDHWC"
        self.input_dim = None

    def build(self, input_shape):
        if len(input_shape) < 5:
            raise ValueError('Inputs to `DepthwiseConv3D` should have rank 5. '
                             'Received input shape:', str(input_shape))
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs to '
                             '`DepthwiseConv3D` '
                             'should be defined. Found `None`.')
        self.input_dim = int(input_shape[channel_axis])

        if self.groups is None:
            self.groups = self.input_dim

        if self.groups > self.input_dim:
            raise ValueError('The number of groups cannot exceed the number of channels')

        if self.input_dim % self.groups != 0:
            raise ValueError('Warning! The channels dimension is not divisible by the group size chosen')

        depthwise_kernel_shape = (self.kernel_size[0],
                                  self.kernel_size[1],
                                  self.kernel_size[2],
                                  self.input_dim,
                                  self.depth_multiplier)

        self.depthwise_kernel = self.add_weight(
            shape=depthwise_kernel_shape,
            initializer=self.depthwise_initializer,
            name='depthwise_kernel',
            regularizer=self.depthwise_regularizer,
            constraint=self.depthwise_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.groups * self.depth_multiplier,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        # Set input spec.
        self.input_spec = InputSpec(ndim=5, axes={channel_axis: self.input_dim})
        self.built = True

    def call(self, inputs, training=None):
        inputs = _preprocess_conv3d_input(inputs, self.data_format)

        if self.data_format == 'channels_last':
            dilation = (1,) + self.dilation_rate + (1,)
        else:
            dilation = self.dilation_rate + (1,) + (1,)

        if self._data_format == 'NCDHW':
            outputs = tf.concat(
                [tf.nn.conv3d(inputs[0][:, i:i+self.input_dim//self.groups, :, :, :], self.depthwise_kernel[:, :, :, i:i+self.input_dim//self.groups, :],
                    strides=self._strides,
                    padding=self._padding,
                    dilations=dilation,
                    data_format=self._data_format) for i in range(0, self.input_dim, self.input_dim//self.groups)], axis=1)

        else:
            outputs = tf.concat(
                [tf.nn.conv3d(inputs[0][:, :, :, :, i:i+self.input_dim//self.groups], self.depthwise_kernel[:, :, :, i:i+self.input_dim//self.groups, :],
                    strides=self._strides,
                    padding=self._padding,
                    dilations=dilation,
                    data_format=self._data_format) for i in range(0, self.input_dim, self.input_dim//self.groups)], axis=-1)

        if self.bias is not None:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)

        return outputs

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            depth = input_shape[2]
            rows = input_shape[3]
            cols = input_shape[4]
            out_filters = self.groups * self.depth_multiplier
        elif self.data_format == 'channels_last':
            depth = input_shape[1]
            rows = input_shape[2]
            cols = input_shape[3]
            out_filters = self.groups * self.depth_multiplier

        depth = conv_utils.conv_output_length(depth, self.kernel_size[0],
                                             self.padding,
                                             self.strides[0])

        rows = conv_utils.conv_output_length(rows, self.kernel_size[1],
                                             self.padding,
                                             self.strides[1])

        cols = conv_utils.conv_output_length(cols, self.kernel_size[2],
                                             self.padding,
                                             self.strides[2])

        if self.data_format == 'channels_first':
            return (input_shape[0], out_filters, depth, rows, cols)

        elif self.data_format == 'channels_last':
            return (input_shape[0], depth, rows, cols, out_filters)

    def get_config(self):
        config = super(DepthwiseConv3D, self).get_config()
        config.pop('filters')
        config.pop('kernel_initializer')
        config.pop('kernel_regularizer')
        config.pop('kernel_constraint')
        config['depth_multiplier'] = self.depth_multiplier
        config['depthwise_initializer'] = initializers.serialize(self.depthwise_initializer)
        config['depthwise_regularizer'] = regularizers.serialize(self.depthwise_regularizer)
        config['depthwise_constraint'] = constraints.serialize(self.depthwise_constraint)
        return config

DepthwiseConvolution3D = DepthwiseConv3D

In [10]:
def model_depthwise(video):
  model_d = Sequential()

  # Adding Alternate convolutional and pooling layers
  model_d.add(DepthwiseConv3D( kernel_size=(10, 3, 3),depth_multiplier=2, strides=(1, 1, 1), padding='same', activation='relu', 
                  input_shape=(29,128,128,1)))
  model_d.add(MaxPooling3D(pool_size=2, strides=(1, 2, 2), padding='same'))

  model_d.add(DepthwiseConv3D( depth_multiplier=4,kernel_size=(5, 3, 3), strides=(3, 1, 1), padding='valid', activation='relu'))
  model_d.add(MaxPooling3D(pool_size=2, strides=(1, 2, 2), padding='same'))

  model_d.add(DepthwiseConv3D(depth_multiplier=8, kernel_size=(5, 3, 3), strides=(3, 1, 1), padding='valid', activation='relu'))
  model_d.add(MaxPooling3D(pool_size=2, strides=(1, 2, 2), padding='same'))

  # A global average pooling layer to get a 1-d vector
  # The vector will have a depth (same as number of elements in the vector) of 256
  model_d.add(GlobalAveragePooling3D())


  # The Global average pooling layer is followed by a fully-connected neural network, with one hidden and one output layer

  # Hidden Layer
  model_d.add(Dense(32, activation='relu'))

  # Output layer
  model_d.add(Dense(2, activation='softmax'))

  #model_d.summary()
  model_d.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model_d.load_weights('/content/Model_depthwise_1_Assault.weights.best.hdf5')
  y=model_d.predict(video)
  y=np.argmax(y[0])
  return y


def model(video):
  model = Sequential()

  # Adding Alternate convolutional and pooling layers
  model.add(Conv3D(filters=16, kernel_size=(10, 3, 3), strides=(1, 1, 1), padding='same', activation='relu', 
                  input_shape=(29,128,128,1)))
  model.add(MaxPooling3D(pool_size=2, strides=(1, 2, 2), padding='same'))

  model.add(Conv3D(filters=64, kernel_size=(5, 3, 3), strides=(3, 1, 1), padding='valid', activation='relu'))
  model.add(MaxPooling3D(pool_size=2, strides=(1, 2, 2), padding='same'))

  model.add(Conv3D(filters=256, kernel_size=(5, 3, 3), strides=(3, 1, 1), padding='valid', activation='relu'))
  model.add(MaxPooling3D(pool_size=2, strides=(1, 2, 2), padding='same'))

  # A global average pooling layer to get a 1-d vector
  # The vector will have a depth (same as number of elements in the vector) of 256
  model.add(GlobalAveragePooling3D())

  # The Global average pooling layer is followed by a fully-connected neural network, with one hidden and one output layer

  # Hidden Layer
  model.add(Dense(32, activation='relu'))

  # Output layer
  model.add(Dense(2, activation='softmax'))

  #model.summary()
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.load_weights('/content/gdrive/My Drive/Security Dataset/Shooting/weights/Model_shooting_1.weights.best.hdf5')
  k=model.predict(video)
  prob=model.predict_proba(video)
  y=np.argmax(k[0])
  #a=np.argmax(k, axis=1)
  return y,prob


In [12]:

#import PILasOPENCV as ImageFont
#from utils import Videos
#font = cv2.FONT_HERSHEY_SIMPLEX 
font =cv2.FONT_HERSHEY_COMPLEX
extract_frames="middle"
writer=None
def _process_video(video):
    """
    Parameters:
        video (Numpy.ndarray):
            Shape = (<No. of frames>, <height>, <width>, <channels>)

            Video whose frames are to be extracted

    Returns:
        Numpy.ndarray
            A tensor (processed video) with shape (<`max_frames`>, <height>, <width>, <channels>)
    """

    total_frames = video.shape[0]
    max_frames=30
    if max_frames <= total_frames:

        if extract_frames == 'first':
            video = video[:max_frames]
        elif extract_frames == 'last':
            video = video[(total_frames - max_frames):]
        elif extract_frames == 'middle':
            # No. of frames to remove from the front
            front = ((total_frames - max_frames) // 2) + 1
            video = video[front:(front + max_frames)]
        else:
            raise ValueError('Invalid value of \'extract_frames\'')

    else:
        raise IndexError(
            'Required number of frames is greater than the total number of frames in the video')

    return video


def read_video(framelist):
        """
        Parameters:
            path (str): Required
                Path of the video to be read

        Returns:
            Numpy.ndarray
                A 5-d tensor with shape (1, <No. of frames>, <height>, <width>, <channels>)
        """

        #cap = FFmpegReader(filename=path)
        list_of_frames = []
        nosfps=30
        target_size=(128,128)
        #self.fps = int(cap.inputfps)                  # Frame Rate
        print(len(framelist))
        for i in range(len(framelist)):
        	frame=framelist[i]
        	temp_image = image.array_to_img(frame)
        	frame = image.img_to_array(temp_image.resize(target_size,Image.ANTIALIAS)).astype('uint8')
        	list_of_frames.append(frame)

        # Shape of each frame -> (<height>, <width>, 3)
        

        temp_video = np.stack(list_of_frames)
        temp_video = rgb2gray(temp_video)
        #temp_video =_process_video(video=temp_video)

        return np.expand_dims(temp_video, axis=0)



cap = cv2.VideoCapture("/content/gdrive/My Drive/Security Dataset/Written USE CASES/shooting_object.avi")
fps=FPS().start()
l=[]
count=0
global sample
wid=cap.get(3)
hig=cap.get(4)
fourcc2 = cv2.VideoWriter_fourcc(*"XVID")
writer2 = cv2.VideoWriter("/content/video/explosion_infer.avi", fourcc2, 30,(int(wid), int(hig)), True)
while cap.isOpened():
  ret, frame= cap.read()
  if ret== True:
    #frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #frame=cv2.resize(frame,(128,128))
    l.append(frame)
    #cv2.imshow("frame", frame)
    
    if writer == None :
        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        writer = cv2.VideoWriter("/content/shooting_final.avi", fourcc, 30,(frame.shape[1], frame.shape[0]), True)
        
    if len(l)>=29:
      sample=read_video(l)
      y,num=model(sample)
      #print(model(sample))
      s=""
      if y == 0:
        s="Normal"
      else:
        s="Shooting"
      
      
      
      for i in l:
        if s=='Shooting':
          i = cv2.putText(i,"SECURITY THREAT : Shooting",(10,50), font, 0.5, (0,0,255), 1,cv2.LINE_AA)
          i = cv2.putText(i,"Assault Probability:"+str(format(num[0][1],".8f")),(10,65), font, 0.5, (0,0,255), 1,cv2.LINE_AA)
          i = cv2.putText(i,"Weapon Detected : Hand Pistol",(10,80), font, 0.5, (0,255,255), 1,cv2.LINE_AA)
          #frame = cv2.putText(frame,"People in the Frame:",(10,95), font, 0.5, (0,255,255), 1,cv2.LINE_AA)
          i = cv2.putText(i,"Crowd Analysis: Less Dense " , (10,110),font, 0.5, (0,255,255), 1,cv2.LINE_AA)
          writer2.write(i)
          cv2.imwrite("/content/infer_frame/frame_"+str(i)+".jpg",frame)
          writer.write(i)
        else:
          i = cv2.putText(i,"SECURITY THREAT : NORMAL",(10,50), font, 0.5, (0,255,255), 1,cv2.LINE_AA)
          i = cv2.putText(i,"Normal Probability:"+str(format(num[0][0],".8f")),(10,65), font, 0.5, (0,255,255), 1,cv2.LINE_AA)
          i = cv2.putText(i,"Weapon Detected : Hand Pistol",(10,80), font, 0.5, (0,255,255), 1,cv2.LINE_AA)
          #frame = cv2.putText(frame,"People in the Frame:",(10,95), font, 0.5, (0,255,255), 1,cv2.LINE_AA)
          i = cv2.putText(i,"Crowd Analysis: Less Dense" , (10,110),font, 0.5, (0,255,255), 1,cv2.LINE_AA)
          
          
          
          writer.write(i)
      
      count+=1
      print("Passing :",count)
      print(s)
      print("Shooting Probability :",format(num[0][1],".8f"))
      print("Normal Probability :",format(num[0][0],".8f")) 
      l=[]
      fps.update()
    
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

fps.stop()
print(fps.fps())
writer.release()
writer2.release()
cap.release()
cv2.destroyAllWindows()

29
Passing : 1
Shooting
Shooting Probability : 1.00000000
Normal Probability : 0.00000000
29
Passing : 2
Normal
Shooting Probability : 0.00000000
Normal Probability : 1.00000000
29
Passing : 3
Shooting
Shooting Probability : 1.00000000
Normal Probability : 0.00000000
29
Passing : 4
Normal
Shooting Probability : 0.00000000
Normal Probability : 1.00000000


KeyboardInterrupt: ignored

In [ ]:
format(4.6335987050611395e-28,".8f")

'0.00000000'

In [ ]:
!mkdir "/content/image_file_infer"

In [9]:
!cp "/content/murder_assault_final.avi" "/content/gdrive/My Drive/Security Dataset/Written USE CASES/"

In [ ]:
!cp "/content/video/explosion_infer.avi" "/content/gdrive/My Drive/Security Dataset/Written USE CASES/"

cp: cannot stat '/content/video/explosion_infer.avi': No such file or directory


In [ ]:
l=os.listdir("/content/")

In [ ]:
k=[]
for i in l:
  if ".jpg" in i:
    k.append(i) 

In [ ]:
for i in os.listdir("/content/"):
  if i in k:
    os.remove("/content/"+i)

In [ ]:
import numpy as np
import cv2

# Create a black image
img = np.zeros((512,512,3), np.uint8)

# Write some Text

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (0,255,255)
lineType               = 2

cv2.putText(img,'Hello World!', 
    bottomLeftCornerOfText, 
    font, 
    fontScale,
    fontColor,
    lineType)

#Display the image
#cv2.imshow("img",img)

#Save image
cv2.imwrite("out.jpg", img)

cv2.waitKey(0)

-1